# ECON240 (Regression), Fall 2021, Problem Set 4
Satej Soman, satej@berkeley.edu

---

# 0 Preliminaries: install packages, etc.

In [2]:
! pip3 install numpy pandas matplotlib statsmodels

In [81]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from matplotlib import pyplot as plt


## common functions (OLS and Bayesian Bootstrap)

In [218]:
# define OLS function to return point estimates and standard errors
def OLS(Y, X, add_constant = True):
    N = len(Y)
    regressors = list(X.columns)
    if add_constant: 
        X, regressors = np.c_[np.ones(X.shape[0]), X], ['intercept'] + regressors
    Z = X.T @ X
    V = np.linalg.inv(Z) # G 
    β = V @ X.T.dot(Y) 
    U = np.linalg.norm(Y - X.dot(β))**2
    σ = np.sqrt(np.diag(V @ Z @ V.T * U / N))
    return pd.DataFrame(data = {
        "coefficient"   : β, 
        "standard error": σ
    },
    index = regressors)

# define OLS function to return point estimates, standard errors, and CI
def OLS_CI(Y, X, add_constant = True, α = 0.05):
    α = min(α, 1 - α)
    N = len(Y)
    regressors = list(X.columns)
    if add_constant: 
        X, regressors = np.c_[np.ones(X.shape[0]), X], ['intercept'] + regressors
    Γ_inv = np.linalg.inv(X.T @ X)
    β = Γ_inv @ X.T.dot(Y) 
    u = Y - X.dot(β)
    Ω = X.T.dot(X) * u.dot(u) / N
    Λ = Γ_inv @ Ω @ Γ_inv.T
    σ = np.sqrt(np.diag(Λ))
    q = stats.norm.ppf(1 - α / 2)
    return pd.DataFrame(data = {
        "coefficient"   : β, 
        "standard error": σ,
        f"[{α/2}"       : β - q * σ, 
        f"{1-α/2}]"     : β + q * σ, 
    }, index = regressors)



,coefficient,standard error,[0.025,0.975]
intercept,8.591573,0.106173,8.383479,8.799668
HGC_Age28,0.162558,0.007845,0.147182,0.177934


---

# 1      Linear regression (Example 1)

## 1. Least squares fit of `LogEarn` on (1, `HGC_Age28`)

In [146]:
# Download the data and restrict observations to ones where earnings > 0.
nsly_URL = "https://github.com/bryangraham/Ec240a/raw/master/Ec240a_Fall2021/Problem_Sets/nlsy79.csv"
nsly79   = pd.read_csv(nsly_URL).query("Earnings > 0")

In [147]:
Y = LogEarn   = np.log(nsly79["Earnings"])
X = HGC_Age28 = nsly79[["HGC_Age28"]]

OLS(Y, X)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3812 is different from 2)

In [136]:
sm.OLS(Y, sm.tools.add_constant(X), hasconst = True).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Earnings   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     428.9
Date:                Sun, 05 Dec 2021   Prob (F-statistic):           4.30e-86
Time:                        15:38:07   Log-Likelihood:                -2398.1
No. Observations:                1906   AIC:                             4800.
Df Residuals:                    1904   BIC:                             4811.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.5916      0.106     80.878      0.000       8.383       8.800
HGC_Age28      0.1626      0.008     20.710      0.000       0.147       0.178
==============================================================================
Omnibus:                     1303.659   Durbin-Watson:                   1.854
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32441.197
Skew:                          -2.856   Prob(JB):                         0.00
Kurtosis:                      22.387   Cond. No.                         74.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**The StatsModels routine and the manually-implemented OLS agree in their point estimates and standard errors.**

## 2. Least squares fit of `LogEarn` on a constant, `HGC_Age28` and `AFQT`

We calculate a long regression (LR) of $Y$ = `LogEarn` on a constant, $X$ = `HGC_Age28` and $W$ = `AFQT`, and compare it to the short regression (SR) of `LogEarn` on a constant and `HGC_Age28`, as well as an auxiliary regression (AR) of `HGC_Age28` on `AFQT`. 

Long regression: $$ Y = \alpha_0 + X^\prime \beta_0 + W^\prime \gamma_0 $$ 
Short regression: $$ Y = X^\prime b_0 $$
Auxiliary regression: $$ W = \omega_0 + X^\prime \Pi_0 $$

Next, to express the SR coefficients in terms of the LR and AR results, construct the error vector in the long regression: $$U = Y - \mathbb{E}^*[ Y\ |\ X, W ]$$

By the Projection Theorem, this vector must be orthogonal to $X$, so:
$$\begin{align*}
\mathbb{E}^*[U\ |\ X ] &= 0 \\ 
\implies \mathbb{E}^*[Y\ |\ X] &= \alpha_0 + X^\prime \beta_0 + (\omega_0 + \Pi_0 X)^\prime \gamma_0 \\
&= \left( \alpha_0 + \omega_0^\prime \gamma_0  \right) + X^\prime(\beta_0 + \Pi_0^\prime \gamma_0) \\ 
\implies b_0 &= \beta_0 + \Pi_0^\prime \gamma_0
\end{align*}$$

We can then verify the coefficients align by running the regressions:

In [137]:
# long, short, and aux regressions
LR = OLS(Y, nsly79[["HGC_Age28", "AFQT"]])
SR = OLS(Y, nsly79[["HGC_Age28"]])
AR = OLS(nsly79["AFQT"], nsly79[["HGC_Age28"]])

print("long :\n", LR, "\n")
print("short:\n", SR, "\n")
print("aux  :\n", AR, "\n")

long :
            coefficient  standard error
intercept     8.915092        0.112372
HGC_Age28     0.108642        0.010345
AFQT          0.007072        0.000903 

short:
            coefficient  standard error
intercept     8.591573        0.106173
HGC_Age28     0.162558        0.007845 

aux  :
            coefficient  standard error
intercept   -45.745421        2.650927
HGC_Age28     7.623701        0.195878 



In [78]:
β = LR["coefficient"]["HGC_Age28"]
γ = LR["coefficient"]["AFQT"]
Π = AR["coefficient"]["HGC_Age28"]
b = SR["coefficient"]["HGC_Age28"]

β + Π*γ, b


(0.16255788018544487, 0.16255788018552053)

**As expected, the short regression coefficients can be expressed in terms of the long regression and auxiliary regression coefficients.**

## 3. Single regression

## 4. Extended regression

We want to fit the model: $ \mathbb{E}^*[\texttt{LogEarn}\,|\, \mathbf{X}] = \alpha_0 + \beta_0 \texttt{HGCAge28} + \gamma_0 \left(\texttt{HGCAge28} \times (\texttt{AFQT - 50})\right) + \delta_0 \texttt{AFQT}$, where $\mathbf{X} = (\texttt{HGCAge28}, \texttt{HGCAge28} \times (\texttt{AFQT - 50}), \texttt{AFQT})$.

### a. *Provide a semi-elasticity interpretation of $\beta_0$.*
Since the LHS is already in log-terms, $\beta_0$ is naturally interpretable 

### b. *Provide a semi-elasticity interpretation of $\beta_0 + \gamma_0 + (\texttt{AFQT} - 50)$.*
### c. *Interpret the null hypothesis $H_0: \gamma_0 = 0$*


In [138]:
# add interaction variable and run OLS
nsly79["HGC_Age28 × (AFQT - 50)"] = nsly79["HGC_Age28"] * (nsly79["AFQT"] - 50)
OLS(Y, nsly79[["HGC_Age28", "AFQT", "HGC_Age28 × (AFQT - 50)"]])

,coefficient,standard error
intercept,8.654594,0.247242
HGC_Age28,0.113838,0.011236
AFQT,0.011238,0.003636
HGC_Age28 × (AFQT - 50),-0.000331,0.000280
